---

### Import Libraries and Dependencies

In [16]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

### Read CSV into Pandas DataFrame

In [17]:

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index
stock_df = pd.read_csv(
    Path("2016_stock_data.csv"), infer_datetime_format=True, parse_dates=True)
#stock_df.reset_index()

stock_df.head()

,Date,AMZN,AAPL,NFLX
0,2016-04-04,593.190002,25.877037,104.349998
1,2016-04-05,586.140015,25.571972,104.940002
2,2016-04-06,602.080017,25.839775,104.830002
3,2016-04-07,591.429993,25.276215,104.449997
4,2016-04-08,594.599976,25.304159,103.809998


### Generate a Dual Moving Average Crossover Trading Signal

In [18]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = stock_df.loc[:, ["Date", "AAPL"]].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Set the short window and long windows
short_window = 5
long_window = 8

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA5"] = signals_df["AAPL"].rolling(window=short_window).mean()
signals_df["SMA8"] = signals_df["AAPL"].rolling(window=long_window).mean()

# Initialize the new `Signal` column
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 (buy) is when the SMA5 is under the SMA10, and
# where 1 (sell) is when the SMA5 is higher (or crosses over) the SMA10
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA5"][short_window:] < signals_df["SMA8"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Print the DataFrame
signals_df.tail()

,AAPL,SMA5,SMA8,Signal,Entry/Exit
Date,,,,,
2016-06-08,23.180901,23.065628,23.163037,1.0,0.0
2016-06-09,23.347248,23.156065,23.142536,0.0,-1.0
2016-06-10,23.155132,23.198706,23.112371,0.0,0.0
2016-06-13,22.806034,23.138260,23.079570,0.0,0.0
2016-06-14,22.834148,23.064693,23.071955,1.0,1.0


### Plot Entry and Exit Points of Dual Moving Average Crossover Trading Strategy

In [19]:
# Visualize exit position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == -1.0]["AAPL"].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == 1.0]["AAPL"].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["AAPL"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA5', 'SMA8']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I         :Curve   [Date]   (AAPL)
   .NdOverlay.I     :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.AAPL.I  :Scatter   [Date]   (AAPL)
   .Scatter.AAPL.II :Scatter   [Date]   (AAPL)

### Backtest the Trading Strategy

In [20]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = -500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 500 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Portfolio Holdings"] = (
    signals_df['AAPL'] * signals_df["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["AAPL"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Portfolio Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
signals_df["Portfolio Daily Returns"] = signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
signals_df["Portfolio Cumulative Returns"] = (
    1 + signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.head()

,AAPL,SMA5,SMA8,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2016-04-04,25.877037,NaN,NaN,0.0,NaN,-0.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-05,25.571972,NaN,NaN,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
2016-04-06,25.839775,NaN,NaN,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2016-04-07,25.276215,NaN,NaN,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2016-04-08,25.304159,25.573832,NaN,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.0,0.0


### Plot Entry/Exit Points of Backtest Results

In [21]:
# Visualize exit position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [22]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Calculate and Assign Portfolio Evaluation Metrics

In [23]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']]
sortino_ratio_df.loc[:,'Downside Returns'] = 0
sortino_ratio_df.head()

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,0.0630962
Cumulative Returns,0.0122911
Annual Volatility,0.0231712
Sharpe Ratio,2.72304
Sortino Ratio,6.09124


### Prepare Trade Evaluation Metrics DataFrame

In [24]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [25]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row["AAPL"]

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row["AAPL"] * row['Entry/Exit Position'])
        exit_share_price = row["AAPL"]
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': "AAPL",
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2016-04-13,2016-04-14,-500.0,26.091280,26.105249,13045.639990,13052.624700,-6.98471
1,AAPL,2016-04-20,2016-05-19,-500.0,24.947865,22.070351,12473.932265,11035.175325,1438.75694
2,AAPL,2016-06-03,2016-06-09,-500.0,22.941923,23.347248,11470.961570,11673.624040,-202.66247


In [26]:
### Create Hvplot Table of Trade Metrics 

In [27]:
price_df = signals_df[["AAPL", 'SMA5', 'SMA8']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [28]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [29]:
# Assemble dashboard visualization
display(Markdown("# Trading Dashboard"))
display(price_chart)
portfolio_evaluation_table + trade_evaluation_table

NameError: name 'Markdown' is not defined